In [1]:
import pandas as pd
import numpy as np

In [2]:
user_engagement = pd.read_csv("takehome_user_engagement.csv")
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [3]:
users = pd.read_csv("takehome_users.csv", encoding = "latin1")
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [4]:
user_engagement["datetime"] = pd.to_datetime(user_engagement.time_stamp)
user_engagement_counts = user_engagement["user_id"].value_counts()
user_engagement_3_or_more = user_engagement[user_engagement["user_id"].isin(user_engagement_counts[user_engagement_counts > 2].index)]
user_engagement_3_or_more.head()

,time_stamp,user_id,visited,datetime
1,2013-11-15 03:45:04,2,1,2013-11-15 03:45:04
2,2013-11-29 03:45:04,2,1,2013-11-29 03:45:04
3,2013-12-09 03:45:04,2,1,2013-12-09 03:45:04
4,2013-12-25 03:45:04,2,1,2013-12-25 03:45:04
5,2013-12-31 03:45:04,2,1,2013-12-31 03:45:04


In [5]:
adopted_users = []
for this_user in user_engagement_3_or_more["user_id"].unique():
    this_users_engagement = user_engagement_3_or_more[user_engagement_3_or_more["user_id"] == this_user]
    this_users_datetime = this_users_engagement["datetime"].reset_index()["datetime"]
    for i in range(len(this_users_datetime) - 2):
        time_interval = this_users_datetime[i + 2] - this_users_datetime[i]
        if time_interval < pd.Timedelta("7 days"):
            adopted_users.append(this_user)
            break

len(adopted_users)

1602

In [6]:
user_adoption_ind = users["object_id"].isin(adopted_users)
pd.crosstab(user_adoption_ind, users["creation_source"], normalize = "columns")

creation_source,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
object_id,,,,,
False,0.833564,0.870005,0.922312,0.859607,0.832491
True,0.166436,0.129995,0.077688,0.140393,0.167509


In [7]:
pd.crosstab(user_adoption_ind, users["opted_in_to_mailing_list"], normalize = "columns")

opted_in_to_mailing_list,0,1
object_id,,
False,0.868088,0.861723
True,0.131912,0.138277


In [8]:
pd.crosstab(user_adoption_ind, users["enabled_for_marketing_drip"], normalize = "columns")

enabled_for_marketing_drip,0,1
object_id,,
False,0.867163,0.862723
True,0.132837,0.137277
